In [66]:
import requests
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
import time
import datetime

In [80]:
def get_track_features(id):
    meta = sp.track(id)
    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    spotify_url = meta['external_urls']['spotify']
    album_cover = meta['album']['images'][0]['url']
    track_info = [name, album, artist, spotify_url, album_cover]
    return track_info

def get_track_ids(time_frame):
    track_ids = []
    for song in time_frame['items']:
        track_ids.append(song['id'])
    return track_ids


In [81]:
client_id = 'client_id' #insert your client id
client_secret = 'client_secret' # insert your client secret id here
redirect_uri = 'http://127.0.0.1:8080'
scope = "user-top-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, 
                                               client_secret=client_secret, 
                                               redirect_uri=redirect_uri, 
                                               scope=scope))

top_tracks_short = sp.current_user_top_tracks(limit=20, offset=0, time_range="short_term")

In [82]:
track_ids = get_track_ids(top_tracks_short)
# loop over track ids 
tracks = []
for i in range(len(track_ids)):
    time.sleep(.5)
    track = get_track_features(track_ids[i])
    tracks.append(track)

In [83]:
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'spotify_url', 'album_cover'])
df.head(10)

,name,album,artist,spotify_url,album_cover
0,Avalanche,That's The Spirit,Bring Me The Horizon,https://open.spotify.com/track/4EyPadLFhtWojU7...,https://i.scdn.co/image/ab67616d0000b273736145...
1,Astronaut In The Ocean,Astronaut In The Ocean,Our Last Night,https://open.spotify.com/track/2qjH6wZfR0g11Zi...,https://i.scdn.co/image/ab67616d0000b273274c95...
2,Retrospection,Macro,Jinjer,https://open.spotify.com/track/01W4AJOJGOWxlN3...,https://i.scdn.co/image/ab67616d0000b2734a6f9e...
3,Cyberpunk 2077 Theme Song (Metal),Cyberpunk 2077 Theme Song (Metal),Artificial Fear,https://open.spotify.com/track/0fyvnfgqC425qIF...,https://i.scdn.co/image/ab67616d0000b273778100...
4,Death Is No Dread Enemy,Every Nothing,Tides Of Man,https://open.spotify.com/track/1aYxXhXvscuchfr...,https://i.scdn.co/image/ab67616d0000b273ae64b3...
5,Happy Song,"Covers, Vol. 4",First to Eleven,https://open.spotify.com/track/3U4DIIMsGZLTSyr...,https://i.scdn.co/image/ab67616d0000b273d678f6...
6,Bad Guy,Bad Guy,Conquer Divide,https://open.spotify.com/track/3UCQwTclMnDYtS0...,https://i.scdn.co/image/ab67616d0000b2731080c4...
7,I WANNA BE YOUR SLAVE,Teatro d'ira - Vol. I,Måneskin,https://open.spotify.com/track/4pt5fDVTg5GhEvE...,https://i.scdn.co/image/ab67616d0000b2735aa050...
8,Hereafter,Holy Hell,Architects,https://open.spotify.com/track/3VGheGbJrEnGvwt...,https://i.scdn.co/image/ab67616d0000b2730f08d4...
9,Someone Like You,The Predator,Ice Nine Kills,https://open.spotify.com/track/4aKOM2dmLTB5U5q...,https://i.scdn.co/image/ab67616d0000b2736c5c33...
